In [1]:
import os,re
import pandas as pd
import jieba
from tqdm import tqdm
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

# load data
## 上個表df

In [2]:
ROOT = "../02preprocess/news_found_keyterms/"
thisFile = os.listdir(ROOT)[-1]
target = ROOT+"{}".format(thisFile)
print(target)
df = pd.read_pickle(target)
content_s = df.content
df

../02preprocess/news_found_keyterms/news_found_keyterms_20190831


,datetime,title,content,link,source,shzs_company_name,szse_company_name,ntusdNegInContent,ntusdPosInContent,finNegInContent,finPosInContent
11279,2013-01-15 13:57:12,A股持仓账户占比现历史新低 散户出逃机构抢...,2012年12月份股指大涨，但统计数据显示越来越多的人选择清仓离场。12月份A股持仓账户减少...,https://www.lj168.com/965427/disnew,lj168,[贵州茅台],[],"[下降, 不可, 不可否认, 不可否认的, 不成熟, 不到, 不明, 不明确, 不够, 否认...",[希望],"[下降, 不可, 不够, 之后, 低, 使, 公司, 力, 否认, 吸引力, 大幅, 容易,...","[上涨, 主动, 使, 吸引力, 大, 好, 完善, 容易, 提高, 收益, 获得, 稳定,..."
11274,2013-01-16 09:01:03,流通十大减负措施出炉 6只仓储物流股直接受...,国务院生产流通环节用水电价格和运营费用，规范和降低农产品市场收费、强化零售商供应商交易监管、...,https://www.lj168.com/965443/disnew,lj168,[],[广发证券],"[下降, 不确, 不确定, 反垄断, 降低, 处置, 缓慢, 垄断, 竞争, 利用, 保守,...","[管理, 改善, 良好]","[下降, 不确定性, 企业, 低, 健全, 公司, 利用, 力, 反垄断, 垄断, 大幅, ...","[优势, 加强, 受益, 大, 好, 好转, 完全, 完善, 完成, 实现, 巨大, 强, ..."
11272,2013-01-16 09:11:45,发改委出重拳 茅台或掉价20%,1月15日，茅台(600519)集团官网发布声明，“国家发改委价格监督检查与反垄断局和贵州省...,https://www.lj168.com/965445/disnew,lj168,[贵州茅台],[],"[反垄断, 否定, 取消, 限定, 最低, 禁止, 违反, 违法, 垄断, 违反, 垄断]",[],"[低, 公司, 反垄断, 取消, 否定, 垄断, 将, 对, 彻底, 会, 有, 有关, 为...",[影响]
11271,2013-01-16 09:14:30,支付卡商五年内或进账60亿,“启动NFC-SWP试点，规模投放NFC-SWP卡，并启动终端定制。”中国电信[4.37 -...,https://www.lj168.com/965446/disnew,lj168,"[中国联通, 招商银行]",[],"[不会, 没有, 尚未, 徘徊, 透露, 超过, 过多, 渗透, 不会, 没有]","[想像, 分工, 合作]","[之后, 低, 佣金, 使, 公司, 力, 功能, 安全, 完成, 实现, 将, 对, 按,...","[上涨, 使, 尽管, 优势, 受益, 合作, 大, 好, 完成, 实现, 成功, 推进, ..."
11270,2013-01-16 09:20:04,四因素提振军工概念 三板块成黑马牧场,昨日，沪深两市双双高开，沪指放量创出近期新高，收报2325.68点，成交1410亿元，放量1...,https://www.lj168.com/965447/disnew,lj168,[华泰证券],[中航飞机],"[不存在, 不会, 不断, 打击, 阻拦, 斗争, 处置, 减少, 超过, 关上, 不会, ...",[增加],"[不及, 之后, 事件, 企业, 使, 保证, 偏离, 停牌, 公司, 公开, 力, 合理,...","[上涨, 使, 保证, 尽管, 利益, 反弹, 受益, 大, 好, 完成, 实现, 强, 推..."
11266,2013-01-17 08:47:08,国家科学技术奖励大会召开在即,“中国梦”对于每一个国人来讲均是为之奋斗的理想，而“中国梦”的实现有赖于经济的长期稳定增长。...,https://www.lj168.com/965977/disnew,lj168,"[中国建筑, 中国中铁, 中国铁建, 大秦铁路]",[潍柴动力],"[人为, 下降, 不断, 未得到, 尚未, 缺陷, 高压, 毫无, 发动, 过多, 疑问, ...","[澄清, 克服, 希望]","[下降, 企业, 低, 充, 克服, 公司, 力, 升级, 即将, 可靠, 围, 安全, 完...","[值得, 伟大, 尽管, 优势, 光荣, 创新, 努力, 可靠, 增强, 梦想, 大, 完全..."
11265,2013-01-17 08:49:35,农产品价格批发指数创新高 养殖类股受益,数据显示，1月16日，农产品批发价格总指数为209.2；“菜篮子”产品批发价格指数为211....,https://www.lj168.com/965978/disnew,lj168,[],[双汇发展],"[不可, 不到, 不能, 不会, 不断, 刺激, 降低, 屠宰, 对手, 担心, 竞争, 不...","[管理, 增加, 地位, 良好]","[不可, 不能, 企业, 低, 使, 保证, 公司, 公害, 力, 功能, 升级, 受到, ...","[上涨, 使, 保证, 价格上涨, 优势, 利益, 努力, 受益, 地位, 大, 好, 完全..."
11263,2013-01-17 08:52:48,信息安全企业分享IPv6千亿市场 相关概念股一...,国家发改委1月16日公布的《关于2012年国家下一代互联网信息安全专项项目产品测试的有关情况...,https://www.lj168.com/965980/disnew,lj168,[中国联通],"[紫光股份, 浪潮信息]","[不足, 不景气, 不景气的, 不会, 没有, 挑战, 要求, 问题, 现实, 透露, 超过...","[增加, 希望]","[不景气, 不足, 企业, 使, 入侵, 公司, 公开, 力, 升级, 受到, 问题, 围,...","[使, 取得, 大, 好, 完善, 完成, 实现, 成功, 推动, 推进, 普及, 最大, ..."
11259,2013-01-17 14:44:13,全国中小企业股份转让系统揭牌运营 15股整装...,全国中小企业股份转让系统揭牌仪式昨日在京举行，非上市公司股份转让的小范围、区域性试点从此将逐...,https://www.lj168.com/966022/disnew,lj168,"[国泰君安, 招商证券, 中信证券]","[广发证券, 长江证券, 西部证券]","[下降, 不大, 不存在, 不快, 不准, 不能, 不会, 不断, 低水平, 抑制, 没有,...","[管理, 增加, 平衡, 合作, 地位, 把握, 良好]","[一种, 下降, 不能, 事件, 交给, 企业, 低, 低迷, 佣金, 使, 允许, 充, ...","[使, 优势, 充分, 创新, 加强, 取得, 受益, 合作, 地位, 报酬, 大, 好, ..."
11255,2013-01-18 09:17:56,中央一号文件即将公布 相关部委已着手落实(...,今年中央一号文件即将正式发布。据记者了解，目前一号文件已经下发到相关部委，相关部委也已开始制...,https://www.lj168.com/966036/disnew,lj168,[招商证券],[],"[下降, 不足, 不断, 拖累, 要求, 高价, 问题, 透露, 超过, 剧烈, 暴跌, 变...","[管理, 增加, 合作, 良好, 信心, 活力]","[下降, 不足, 企业, 使, 保证, 公司, 剥离, 力, 即将, 问题, 大幅, 安全,...","[上涨, 使, 保证, 价格上涨, 优势, 利益, 加强, 受益, 合作, 增强, 大, 好..."


## 停用詞

In [3]:
sentDictFolder = "../00ref/stopwords//"
allFile = os.listdir(sentDictFolder)
stopWord_set = set()

for afile in allFile:
    target = "{}{}".format(sentDictFolder,afile)
    with open(target,encoding = "utf8") as f:
        notList = f.readlines()
        strlist = map(str.strip,notList[1:])
        stopWord_set.update(strlist)
        
stopWord_set = list(stopWord_set)
allFile

['中文停用詞.txt', '哈工大停用詞.txt', '四川大停用詞.txt', '百度停用詞.txt']

# 塞進 vectorizer 之前的前處理

In [4]:
tokenizedContent_s=content_s.apply(str.strip).progress_apply(jieba.cut)
removeDot_s = tokenizedContent_s.progress_apply(lambda aList:[n.replace(".","") for n in aList])
cleanedContent_s = removeDot_s.progress_apply(lambda aList : [n.strip() for n in aList if ((not bool(re.search(r'\d', n))) and (not n in stopWord_set))])
joinedTokens_s = cleanedContent_s.progress_apply(" ".join)
joinedTokens_s.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Evan\AppData\Local\Temp\jieba.cache
Loading model cost 1.145 seconds.
Prefix dict has been built succesfully.


11279    年 月份 股指 大涨 统计数据 显示 越来越 选择 清仓 离场 月份 A股 持仓 账户 减少...
11274    国务院 生产 流通 环节 水电 价格 运营 费用 规范 降低 农产品 市场 收费 强化 零售...
11272    月 日 茅台 集团 官网 发布 声明 国家 发改委 价格 监督 检查 反垄断 局 贵州省 物...
11271    启动 NFC SWP 试点 规模 投放 NFC SWP 卡 启动 终端 定制 中国电信  董...
11270    昨日 沪 深 两市 双双 高开 沪 指 放量 创出 近期 新高 收报 点 成交 亿元 放量 ...
Name: content, dtype: object

In [5]:
df['joinedTokens_s'] = joinedTokens_s
joinedTokens_s

11279    年 月份 股指 大涨 统计数据 显示 越来越 选择 清仓 离场 月份 A股 持仓 账户 减少...
11274    国务院 生产 流通 环节 水电 价格 运营 费用 规范 降低 农产品 市场 收费 强化 零售...
11272    月 日 茅台 集团 官网 发布 声明 国家 发改委 价格 监督 检查 反垄断 局 贵州省 物...
11271    启动 NFC SWP 试点 规模 投放 NFC SWP 卡 启动 终端 定制 中国电信  董...
11270    昨日 沪 深 两市 双双 高开 沪 指 放量 创出 近期 新高 收报 点 成交 亿元 放量 ...
11266    中国 梦 国人 均 奋斗 理想 中国 梦 有赖于 经济 长期 稳定增长 经济 转型期 科技 ...
11265    数据 显示 月 日 农产品 批发价格 总 指数 菜篮子 产品 批发价格 指数 均 创 历史 ...
11263    国家 发改委 月 日 公布 年 国家 下一代 互联网 信息安全 专项 项目 产品 测试 情况...
11259    全国 中小企业 股份 转让 系统 揭牌 仪式 昨日 在京举行 非上市 公司 股份 转让 区域...
11255    中央 一号 文件 正式 发布 记者 一号 文件 下发 相关 部委 相关 部委 制定 落实 一...
11251    发稿 时 博时 工 银瑞信 基金 公司 率先 公布 旗下 基金 年 四季 报 披露 季报 基...
11247    近期 医药 生物 崛起 无疑 本轮 市场 反弹 注入 新 做 动能 经历 一个多月 逼空 走...
11243    年度 国家 科技 奖励 大会 落下 帷幕 科技 创新 红利 持续增长 A股 上市公司 占有 ...
11242    大小 非 减持 QFII 席位 借道 吸筹 合力 推动 大宗 交易 活跃度 走高 上周 大宗...
11236    基金 公司 金融 地产 食品饮料 三大 行业 配置 翻天覆地 变化          大成 ...
11235    深圳 证券 信息 有限公司 数据中心 统计 截止 月 日 融资 融券 余额 突破 亿元 大关...
11233    昨日 沪 深 两市 延续 原有 多头 格局 市场 多空 分歧 日益加剧 背景 受 国务院 发.

# 塞進 vectorizer

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.8)

tfidf = vectorizer.fit(joinedTokens_s)
train_features = vectorizer.transform(joinedTokens_s)

idf = vectorizer.idf_
term = vectorizer.get_feature_names()
idfDict = dict(zip(term, idf))

In [61]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=None,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [68]:
import pickle
pickle.dump(tfidf, open("vectorizer_model/tfidfVectorizer_{}.pickle".format(str(date.today())), "wb"))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=None,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

# 計算他的情緒因子

In [52]:
# from sklearn import preprocessing
ntusdNegInContent = df.ntusdNegInContent
NegCountMean = ntusdNegInContent.apply(len).mean()
NegCountStd = ntusdNegInContent.apply(len).std()
NegCountNorm = (ntusdNegInContent.apply(len)-NegCountMean)/NegCountStd

ntusdPosInContent = df.ntusdPosInContent
PosCountMean = ntusdPosInContent.apply(len).mean()
PosCountStd = ntusdPosInContent.apply(len).std()
PosCountNorm = (ntusdPosInContent.apply(len)-PosCountMean)/PosCountStd

df['sentimentScore'] = PosCountNorm-NegCountNorm
df['sentimentScore'].head()

11279   -1.301306
11274    0.553302
11272   -0.873620
11271    0.756551
11270   -0.623811
Name: sentimentScore, dtype: float64

In [53]:
ntusdNegInContent = df.finNegInContent
NegCountMean = ntusdNegInContent.apply(len).mean()
NegCountStd = ntusdNegInContent.apply(len).std()
NegCountNorm = (ntusdNegInContent.apply(len)-NegCountMean)/NegCountStd

ntusdPosInContent = df.finPosInContent
PosCountMean = ntusdPosInContent.apply(len).mean()
PosCountStd = ntusdPosInContent.apply(len).std()
PosCountNorm = (ntusdPosInContent.apply(len)-PosCountMean)/PosCountStd

df['finSentimentScore'] = PosCountNorm-NegCountNorm
df['finSentimentScore'].head()


11279   -0.137733
11274    0.568287
11272   -1.002880
11271    1.269953
11270    0.692657
Name: finSentimentScore, dtype: float64

In [54]:
from datetime import date
hybridSentimentScore = df['finSentimentScore']*0.6+df['sentimentScore']*0.4
df['hybridSentimentScore'] =hybridSentimentScore
df.sort_values(by = 'hybridSentimentScore',ascending=False).content
df.to_pickle('./news_scored/news_scored_{}'.format(date.today().strftime("%Y%m%d")))

In [55]:
superGoodTokens = df.sort_values(by = 'hybridSentimentScore',ascending=False)[:int(len(df)*0.05)].joinedTokens_s
superGoodVectors = vectorizer.transform(superGoodTokens)
superGoodVectors.shape

(441, 26267)

In [56]:
import numpy as np
tfidfSum_array = np.asarray(np.sum(superGoodVectors,axis = 0))[0]
superGoodTerms = np.asarray(term)[np.argsort(tfidfSum_array)][-100:]
superGoodTerms

array(['核心', '提供', '资产', '超过', '机会', 'a股', '系统', '投资者', '股东', '信息', '军工',
       '管理', '工作', '设备', '汽车', '规模', '产业链', '上半年', '网络', '工业', '生产', '全球',
       '进一步', '一带', '转型', '数据', '健康', '研发', '消费', '农业', '布局', '需求', '金融',
       '城市', '体育', '收入', '机构', '增持', '医疗', '优势', '加快', '资金', '资本', '规划',
       '能力', '智慧', '持续', '受益', '平台', '制造', '净利润', '关注', '政策', '预计', '重点',
       '项目', '板块', '推动', '国际', '相关', '集团', '个股', '国内', '我国', '上市公司', '战略',
       '经济', '机器人', '有望', '科技', '未来', '国家', '智能', '提升', '合作', '万元', '创新',
       '推进', '互联网', '证券', '技术', '业绩', '服务', '同比', '改革', '产品', '股份', '投资',
       '领域', '亿元', '业务', '中国', '建设', '产业', '企业', '增长', '行业', '市场', '发展',
       '公司'], dtype='<U13')

In [58]:
for aterm in superGoodTerms:
    print(aterm,idfDict[aterm])

核心 2.664991579609432
提供 2.4881741648074382
资产 2.301513482380304
超过 1.8952160812569703
机会 2.5702259138397974
a股 1.9854090380836253
系统 3.025094119020432
投资者 2.038060974038811
股东 2.0281929964622414
信息 2.325529804139853
军工 4.094105937912251
管理 2.340134874745852
工作 2.507936652417505
设备 2.9506796334543486
汽车 3.0199571214225154
规模 2.274812368490876
产业链 2.895648314274425
上半年 2.751241134719696
网络 3.1619418568818056
工业 3.163912300869104
生产 2.4871711556962692
全球 2.640737211128039
进一步 2.2460449033155543
一带 4.596632758863547
转型 2.9169905877561977
数据 1.8916174614564905
健康 3.23756521044387
研发 2.7676942863937377
消费 2.97650834933306
农业 4.017144896776123
布局 2.6136384067751104
需求 2.472739454612877
金融 2.6806829094041076
城市 3.2798479316032476
体育 5.010396669786406
收入 2.313660228584469
机构 2.1633094266669857
增持 3.24609404679139
医疗 3.485699985807396
优势 2.7096883368052205
加快 2.8608170792522616
资金 1.8811720668708516
资本 2.3646652330834543
规划 3.057369052962229
能力 2.406319059067174
智慧 3.90555383136766
持续 1.99700867